We are using a 1mil-tuple sample** of the iowa liquor sales data for this assignment, with column names and datatype (in format of [name] [datatype]) as:

  - address char(256),
  - bottle_volume_ml integer,
  - category char(256),
  - category_name char(256),
  - city char(256),
  - county char(256),
  - county_number char(256),
  - date date,
  - im_desc char(256),
  - invoice_line_no char(256),
  - itemno integer,
  - name char(256),
  - pack integer,
  - sale_bottles integer,
  - sale_dollars double precision,
  - sale_gallons double precision,
  - sale_liters double precision,
  - state_bottle_cost double precision,
  - state_bottle_retail double precision,
  - store integer,
  - store_location char(256),
  - store_location_address char(256),
  - store_location_city char(256),
  - store_location_zip char(256),
  - vendor_name char(256),
  - vendor_no integer,
  - zipcode text
  
A description of column names can be found here: https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy.

You may run this query to have some ideas on the schema you are dealing with:

In [3]:
%%sql
select * from iowa limit 1  ;

1 rows affected.


,address,bottle_volume_ml,category,category_name,city,county,county_number,date,im_desc,invoice_line_no,...,sale_liters,state_bottle_cost,state_bottle_retail,store,store_location_address,store_location_city,store_location_zip,vendor_name,vendor_no,zipcode
0,5491 Business Hwy 151 ...,1750,1031100,American Vodkas ...,Marion ...,LINN ...,57,2018-04-09,Titos Handmade Vodka ...,INV-11371200038 ...,...,10.5,19.0,28.5,3868,5491 Business Hwy 151 ...,Marion ...,52302 ...,FIFTH GENERATION INC ...,301,52302


__Disclaimer: this course does not condone drinking, we are using this dataset because it is a common format for a sales transaction log in a silghtly more accessible domain than typical bank transactions__.

## Part I: SQL, the sequel

### Jupyter Notes: _Read these carefully_

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that you run the final cell to submit your results**
  * you can press shift+enter to execute to code in the cell that your cursor is in.
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_. Please wait for the execution to complete
    * **If the cell is hanging- i.e. running for too long: you can restart the kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute cells from the top
* _Have fun!_

__Now, please write the SQL query for each of the following questions.__

__Fill answers according to instruction in each question. Make sure to submit your query also (instruction given).__

Note: Some queries will take a few minutes to run on your virtual machine. If your query is running for more than ~10 minutes, you either did something wrong, 
or you may need to create tables with your temp tables to run the query more efficiently. 

For example: If you use the results of a sub-query multiple times, it 
can help to use SELECT ... INTO ... then reference that table.


### __(2 points) Q1.1: Which store had the second most sales in terms of total records in the table?__
__You will receive ZERO if you use offset, but you could use offset to verify your answers. Notice that below are some follow ups for this question, so make sure you got the correct answer.__

In [202]:
%%sql
with t1 as(select store,count(*)as total_record from
 iowa group by store order by total_record desc)

select t1.store,t1.total_record from t1 where 
t1.total_record< (select max(t1.total_record) from t1) limit 1;




1 rows affected.


,store,total_record
0,2190,7802


In [203]:
# Please replace None with the result store and record number here
# submit query in string format
q1 = {
    'store': 2190,
    'count': 7802,
    'query': 'with t1 as(select name,count(*)as total_record from iowa group by name order by total_record desc) select t1.name,t1.total_record from t1 where t1.total_record< (select max(t1.total_record) from t1) limit 1;'
}

### __(2 points) Q1.2: At the store with the second most total records (you may use the answer for Q1.1), what was the vendor number with most sale records? (The vendor that has the most records in the table?)__

In [91]:
%%sql
select vendor_no,count(*) as total from iowa where store=2190  group by vendor_no order by total desc limit 1;



1 rows affected.


,vendor_no,total
0,260,1393


### __(2 points) Q1.3: For the store 2130, 2152, 2190 and 2191, write a query to find the most purchased sale_bottles one time (for each row, we regard it as one time). __

__The order of returned tuples (store, sale_bottles) should be from 2130 to 2191. You will receive ZERO if you use group by or any aggregations.__

In [8]:
%%sql
select store,max(sale_bottles) as most_purchased from iowa where store=2130 or store=2152 or store=2190 or store=2191  group by store order by store;



4 rows affected.


,store,most_purchased
0,2130,600
1,2152,36
2,2190,480
3,2191,540


### __(3 points) Q1.4: This problem has two steps, you only need to return the value from second step.__

__Compute the set of all liquors with the characters "tequila" (in lowercase) in its description (`im_desc`). The attribute `itemno` is the one that can serve as a unique identifier for a specific liquor.__

__Return the count of all distinct value of zipcode for all liquor stores that sold at least two of every tequila as defined in the previous sentence.__

(Optional: It is possible to rewrite the query in a form that can execute quite fast, but it will require you to create some temporary tables and/or experiment with alternate forms to express the problem. If you find the solution, feel free to submit it!)

In [217]:
%%sql
select distinct itemno
from iowa where im_desc LIKE '%tequila%'

2 rows affected.


,itemno
0,57148
1,57158


In [233]:
%%sql
with a1 as(select store,itemno,sale_bottles,zipcode from iowa 
      where itemno=57148 or itemno=57158
      order by store,itemno),

a2 as(select store,count(distinct itemno)as item from a1
group by store),

store as(select store from a2 where item=2),

a3 as(select * from a1 where sale_bottles>=2
      and 
      store in (select * from store))
 
select count(distinct zipcode) from a3 


1 rows affected.


,count
0,115


### __ (4 points) Q1.5:  __Compute the median itemno for the store you get in Q1.1

Let's take the store 4396 for example, if you run the query below, you would get the itemno and how many times each itemno showed.

In [12]:
%%sql
select itemno, count(1) as count
from iowa
where store = 4396
group by itemno
order by itemno

65 rows affected.


,itemno,count
0,8828,12
1,11296,3
2,11776,4
3,11777,2
4,11788,14
5,12408,5
6,12888,1
7,15626,1
8,16518,4
9,16987,4


Therefore, the median is computed as the median of the array \[8828, 8828, ... (totally 12 times of 8828), 11296, 11296, 11296, 11776, 11776, 11776, 11776, ..., 89196, 89196\].

__you are asked to write a query to compute the median of the store that you got from Q1.1. Hint: you might want to substitute '4396' with the result of Q1.1 and use that as a temporary table (with clause, subquery or view) first and then see how to compute the median.__

In [235]:
%%sql

with t1 as(select itemno, count(itemno) as count_no         
from iowa
where store = 2190
group by itemno
order by itemno)
select itemno from iowa where store=2190 
order by itemno
offset  (select sum(count_no) from t1)/2
limit 1;




1 rows affected.


,itemno
0,41078
